In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import cupy as cp

from runningstats import StatsTracker

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Check

In [2]:
np.random.seed(0)
sample_size=3000
samples = cp.random.normal(size=(3, 3, sample_size))

In [4]:
tracker = StatsTracker((3,3), device="gpu")
for j in range(sample_size):
    tracker.push(samples[:,:,j])

In [5]:
print(tracker.sample_size())

3000


In [6]:
print(tracker.mean())
print(cp.mean(samples, axis=2))

[[ 0.00302671 -0.0036464  -0.01954829]
 [-0.0033383  -0.0287091   0.00596103]
 [-0.01644524  0.01051236  0.00400089]]
[[ 0.00302671 -0.0036464  -0.01954829]
 [-0.0033383  -0.0287091   0.00596103]
 [-0.01644524  0.01051236  0.00400089]]


In [7]:
print(tracker.variance())
print(cp.var(samples, axis=2, ddof=1))

[[0.97490192 1.01762314 0.97512216]
 [1.00866004 1.01647694 0.99555115]
 [1.01208155 0.99239737 0.98558975]]
[[0.97490192 1.01762314 0.97512216]
 [1.00866004 1.01647694 0.99555115]
 [1.01208155 0.99239737 0.98558975]]


In [8]:
print(tracker.stdev())
print(cp.std(samples, axis=2, ddof=1))

[[0.98737122 1.00877309 0.98748274]
 [1.00432069 1.00820481 0.9977731 ]
 [1.00602264 0.99619143 0.99276873]]
[[0.98737122 1.00877309 0.98748274]
 [1.00432069 1.00820481 0.9977731 ]
 [1.00602264 0.99619143 0.99276873]]


In [11]:
print(tracker.skewness())
print(skew(samples.get(), axis=2))

[[-0.00940243  0.04678884  0.03011636]
 [-0.05063272 -0.08082565  0.00857613]
 [-0.07033733  0.01638844  0.01006242]]
[[-0.00940243  0.04678884  0.03011636]
 [-0.05063272 -0.08082565  0.00857613]
 [-0.07033733  0.01638844  0.01006242]]


In [12]:
print(tracker.kurtosis())
print(kurtosis(samples.get(), axis=2))

[[-0.05406281  0.00125444 -0.06613311]
 [ 0.17673225  0.22061916 -0.18343131]
 [ 0.06036077 -0.00519771 -0.03522557]]
[[-0.05406281  0.00125444 -0.06613311]
 [ 0.17673225  0.22061916 -0.18343131]
 [ 0.06036077 -0.00519771 -0.03522557]]


# C++ source

In [ ]:
# #ifndef RUNNINGSTATS_H
# #define RUNNINGSTATS_H

# class RunningStats
# {
# public:
#     RunningStats();
#     void Clear();
#     void Push(double x);
#     long long NumDataValues() const;
#     double Mean() const;
#     double Variance() const;
#     double StandardDeviation() const;
#     double Skewness() const;
#     double Kurtosis() const;

#     friend RunningStats operator+(const RunningStats a, const RunningStats b);
#     RunningStats& operator+=(const RunningStats &rhs);

# private:
#     long long n;
#     double M1, M2, M3, M4;
# };

# #endif

# And here is the implementation file RunningStats.cpp.

# #include "RunningStats.h"
# #include <cmath>
# #include <vector>

# RunningStats::RunningStats() 
# {
#     Clear();
# }

# void RunningStats::Clear()
# {
#     n = 0;
#     M1 = M2 = M3 = M4 = 0.0;
# }

# void RunningStats::Push(double x)
# {
#     double delta, delta_n, delta_n2, term1;

#     long long n1 = n;
#     n++;
#     delta = x - M1;
#     delta_n = delta / n;
#     delta_n2 = delta_n * delta_n;
#     term1 = delta * delta_n * n1;
#     M1 += delta_n;
#     M4 += term1 * delta_n2 * (n*n - 3*n + 3) + 6 * delta_n2 * M2 - 4 * delta_n * M3;
#     M3 += term1 * delta_n * (n - 2) - 3 * delta_n * M2;
#     M2 += term1;
# }

# long long RunningStats::NumDataValues() const
# {
#     return n;
# }

# double RunningStats::Mean() const
# {
#     return M1;
# }

# double RunningStats::Variance() const
# {
#     return M2/(n-1.0);
# }

# double RunningStats::StandardDeviation() const
# {
#     return sqrt( Variance() );
# }

# double RunningStats::Skewness() const
# {
#     return sqrt(double(n)) * M3/ pow(M2, 1.5);
# }

# double RunningStats::Kurtosis() const
# {
#     return double(n)*M4 / (M2*M2) - 3.0;
# }

# RunningStats operator+(const RunningStats a, const RunningStats b)
# {
#     RunningStats combined;
    
#     combined.n = a.n + b.n;
    
#     double delta = b.M1 - a.M1;
#     double delta2 = delta*delta;
#     double delta3 = delta*delta2;
#     double delta4 = delta2*delta2;
    
#     combined.M1 = (a.n*a.M1 + b.n*b.M1) / combined.n;
    
#     combined.M2 = a.M2 + b.M2 + 
#                   delta2 * a.n * b.n / combined.n;
    
#     combined.M3 = a.M3 + b.M3 + 
#                   delta3 * a.n * b.n * (a.n - b.n)/(combined.n*combined.n);
#     combined.M3 += 3.0*delta * (a.n*b.M2 - b.n*a.M2) / combined.n;
    
#     combined.M4 = a.M4 + b.M4 + delta4*a.n*b.n * (a.n*a.n - a.n*b.n + b.n*b.n) / 
#                   (combined.n*combined.n*combined.n);
#     combined.M4 += 6.0*delta2 * (a.n*a.n*b.M2 + b.n*b.n*a.M2)/(combined.n*combined.n) + 
#                   4.0*delta*(a.n*b.M3 - b.n*a.M3) / combined.n;
    
#     return combined;
# }

# RunningStats& RunningStats::operator+=(const RunningStats& rhs)
# { 
#         RunningStats combined = *this + rhs;
#         *this = combined;
#         return *this;
# }